# Multi-Class EEG Digit Classification Analysis

This notebook provides an interactive analysis of the multi-class EEG classification project, extending from binary (6 vs 9) to full 10-digit classification (0-9).

## 1. Setup and Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import torch
import sys
import os

# Add src directory to path
sys.path.append('../src')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📚 Libraries imported successfully")

## 2. Data Loading and Exploration

In [ ]:
# Load multi-class data
from multiclass_data_loader import load_multiclass_data, preprocess_multiclass_data

# Load data
data_path = "../data/Data/EP1.01.txt"
X, y, metadata = load_multiclass_data(data_path, max_samples_per_digit=100)

print(f"Loaded {len(X)} samples")
print(f"Classes: {np.unique(y)}")
print(f"Class distribution: {np.bincount(y)}")

In [ ]:
# Visualize class distribution
plt.figure(figsize=(10, 6))
unique, counts = np.unique(y, return_counts=True)
bars = plt.bar([f'Digit {i}' for i in unique], counts, color='skyblue', edgecolor='black')

# Add value labels
for bar, count in zip(bars, counts):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 1,
             str(count), ha='center', va='bottom')

plt.xlabel('Digit Class')
plt.ylabel('Number of Samples')
plt.title('Multi-Class Dataset Distribution')
plt.grid(True, alpha=0.3, axis='y')
plt.show()

## 3. Complexity Analysis: Binary vs Multi-Class

In [ ]:
# Theoretical complexity comparison
complexity_data = {
    'Metric': ['Number of Classes', 'Decision Boundaries (OvR)', 'Decision Boundaries (OvO)', 
               'Random Baseline', 'Expected Accuracy', 'Confidence > 0.9 (%)'],
    'Binary (6 vs 9)': [2, 1, 1, '50%', '80-85%', '80-90%'],
    'Multi-class (0-9)': [10, 9, 45, '10%', '25-40%', '5-15%']
}

df_complexity = pd.DataFrame(complexity_data)
print("Complexity Comparison:")
print(df_complexity.to_string(index=False))

In [ ]:
# Visualize decision boundary complexity
classes = np.arange(2, 11)
ovr_boundaries = classes - 1
ovo_boundaries = classes * (classes - 1) // 2

plt.figure(figsize=(10, 6))
plt.plot(classes, ovr_boundaries, 'o-', label='One-vs-Rest', linewidth=2, markersize=8)
plt.plot(classes, ovo_boundaries, 's-', label='One-vs-One', linewidth=2, markersize=8)

plt.xlabel('Number of Classes')
plt.ylabel('Number of Decision Boundaries')
plt.title('Decision Boundary Complexity Growth')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')

# Highlight binary vs 10-class
plt.axvline(x=2, color='green', linestyle='--', alpha=0.7, label='Binary (Current)')
plt.axvline(x=10, color='red', linestyle='--', alpha=0.7, label='10-class (Target)')
plt.legend()
plt.show()

## 4. Sample Data Analysis

In [ ]:
# Preprocess data
X_processed, y_processed = preprocess_multiclass_data(X, y)

# Visualize sample signals for different digits
fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('Sample EEG Signals for Each Digit', fontsize=16)

for digit in range(10):
    row = digit // 5
    col = digit % 5
    
    # Find first sample of this digit
    digit_indices = np.where(y_processed == digit)[0]
    if len(digit_indices) > 0:
        sample_idx = digit_indices[0]
        signal = X_processed[sample_idx]
        
        axes[row, col].plot(signal, linewidth=1)
        axes[row, col].set_title(f'Digit {digit}')
        axes[row, col].set_xlabel('Time')
        axes[row, col].set_ylabel('Amplitude')
        axes[row, col].grid(True, alpha=0.3)
    else:
        axes[row, col].text(0.5, 0.5, f'No data\nfor digit {digit}', 
                           ha='center', va='center', transform=axes[row, col].transAxes)

plt.tight_layout()
plt.show()

## 5. Inter-Class Similarity Analysis

In [ ]:
# Calculate pairwise correlations between digit classes
digit_means = []
digit_labels = []

for digit in range(10):
    digit_samples = X_processed[y_processed == digit]
    if len(digit_samples) > 0:
        digit_mean = np.mean(digit_samples, axis=0)
        digit_means.append(digit_mean)
        digit_labels.append(digit)

# Calculate correlation matrix
correlation_matrix = np.corrcoef(digit_means)

# Visualize correlation matrix
plt.figure(figsize=(10, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.3f', 
            xticklabels=[f'Digit {i}' for i in digit_labels],
            yticklabels=[f'Digit {i}' for i in digit_labels],
            cmap='coolwarm', center=0)
plt.title('Inter-Class Correlation Matrix\n(Higher values indicate more similar EEG patterns)')
plt.tight_layout()
plt.show()

# Find most and least similar digit pairs
np.fill_diagonal(correlation_matrix, 0)  # Remove self-correlations
max_corr_idx = np.unravel_index(np.argmax(correlation_matrix), correlation_matrix.shape)
min_corr_idx = np.unravel_index(np.argmin(correlation_matrix), correlation_matrix.shape)

print(f"Most similar digits: {digit_labels[max_corr_idx[0]]} and {digit_labels[max_corr_idx[1]]} (correlation: {correlation_matrix[max_corr_idx]:.3f})")
print(f"Least similar digits: {digit_labels[min_corr_idx[0]]} and {digit_labels[min_corr_idx[1]]} (correlation: {correlation_matrix[min_corr_idx]:.3f})")

## 6. Expected Performance Analysis

In [ ]:
# Literature comparison
literature_data = {
    'Study': ['Kaongoen & Jo (2017)', 'Bird et al. (2019)', 'Spampinato et al. (2017)', 
              'This Work (Target)', 'Random Baseline'],
    'Accuracy (%)': [31.2, 28.7, 40.0, 35.0, 10.0],
    'Method': ['CNN', 'Deep Learning', 'CNN + Transfer', 'Ensemble', 'Random']
}

df_literature = pd.DataFrame(literature_data)

plt.figure(figsize=(12, 6))
bars = plt.bar(df_literature['Study'], df_literature['Accuracy (%)'], 
               color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFA07A'])

# Add value labels
for bar, acc in zip(bars, df_literature['Accuracy (%)']):
    plt.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
             f'{acc}%', ha='center', va='bottom', fontweight='bold')

plt.ylabel('Accuracy (%)')
plt.title('Multi-Class EEG Classification: Literature Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

print("Literature Summary:")
print(df_literature.to_string(index=False))

## 7. Confidence Distribution Simulation

In [ ]:
# Simulate expected confidence distributions
np.random.seed(42)

# Binary task confidence (from actual results)
binary_confidence = np.random.beta(8, 2, 1000)  # High confidence distribution

# Multi-class confidence (expected)
multiclass_confidence = np.random.beta(2, 3, 1000)  # Lower confidence distribution

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(binary_confidence, bins=30, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.title(f'Binary Classification (6 vs 9)\nMean: {binary_confidence.mean():.3f}')
plt.axvline(binary_confidence.mean(), color='red', linestyle='--', label='Mean')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(multiclass_confidence, bins=30, alpha=0.7, color='orange', edgecolor='black')
plt.xlabel('Confidence')
plt.ylabel('Frequency')
plt.title(f'Multi-Class (0-9) - Expected\nMean: {multiclass_confidence.mean():.3f}')
plt.axvline(multiclass_confidence.mean(), color='red', linestyle='--', label='Mean')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Binary task - High confidence (>0.9): {np.sum(binary_confidence > 0.9)/len(binary_confidence)*100:.1f}%")
print(f"Multi-class - High confidence (>0.9): {np.sum(multiclass_confidence > 0.9)/len(multiclass_confidence)*100:.1f}%")

## 8. Research Questions and Hypotheses

### Key Research Questions:

1. **Scalability**: How does classification difficulty scale with the number of classes?
2. **Discriminability**: Which digits are most/least distinguishable in EEG signals?
3. **Hierarchical Approaches**: Can hierarchical classification improve performance?
4. **Attention Mechanisms**: What is the role of attention in multi-class EEG classification?
5. **Confidence Calibration**: How does confidence distribution change with task complexity?

### Hypotheses:

1. **H1**: Multi-class accuracy will be 25-40% (vs 10% random baseline)
2. **H2**: Confidence distribution will be more realistic (mean ~0.4 vs ~0.9 for binary)
3. **H3**: Hierarchical ensemble will outperform flat multi-class approaches
4. **H4**: Attention mechanisms will be crucial for identifying discriminative temporal patterns
5. **H5**: Some digit pairs (e.g., 6-8, 3-8) will show higher confusion rates

## 9. Next Steps

### Immediate Tasks:
1. Run the complete multi-class experiment: `python ../src/run_multiclass_experiment.py`
2. Analyze results and compare with literature
3. Develop advanced ensemble methods
4. Generate publication-quality figures

### Publication Strategy:
1. **Target Journals**: IEEE TNSRE, Journal of Neural Engineering, Frontiers in Neuroscience
2. **Key Contributions**: Hierarchical ensemble, confidence analysis, scalability study
3. **Timeline**: 3-6 months for complete study and manuscript preparation